Acquire Stock Data

In [1]:
%pip install yfinance pandas xgboost scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---

In [7]:
import yfinance as yf
import pandas as pd

df = yf.download("AAPL", period="5y")

C:\Users\zihan\AppData\Local\Temp\ipykernel_9888\788186488.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("AAPL", period="5y")
[*********************100%***********************]  1 of 1 completed


In [19]:
%pip install praw transformers

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.3 MB 245.4 kB/s eta 0:00:44
   

In [31]:
import praw

reddit = praw.Reddit(
    client_id="P6_n0-NZ-6W6y24ieL3hsA",
    client_secret="z-YjU9NoZOaV8QUx-xrrx923qFVWvQ",
    user_agent="reddit scraper v1.0 by u/Apprehensive_Fig6885"
)

query = '(title:"NVDA" OR selftext:"NVDA") AND (title:stock OR title:share OR title:earnings OR title:market)'
finance_keywords = [
    "stock", "share", "ticker", "dividend", "price", "earnings",
    "revenue", "guidance", "options", "puts", "calls", "market",
    "invest", "investing", "hold", "buy", "sell", "portfolio"
]
subreddit = reddit.subreddit("all")
limit = 1000

def is_finance_related(text, keywords):
    text_lower = text.lower()
    return any(word in text_lower for word in keywords)

records = []

for submission in subreddit.search(query, sort="relevance", time_filter="month", limit=limit):
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
        clean_body = comment.body.replace("\n", " ").strip()

        if (
            is_finance_related(clean_body, finance_keywords) and
            len(clean_body.split()) > 5 and
            comment.author not in ["AutoModerator", None]
        ):
            records.append({
                "Post Title": submission.title,
                "Post ID": submission.id,
                "Comment ID": comment.id,
                "Comment": clean_body,
                "Score": comment.score,
                "Author": str(comment.author),
                "Created UTC": pd.to_datetime(comment.created_utc, unit='s')
            })

df_comments = pd.DataFrame(records)

In [20]:
%pip install torch

Defaulting to user installation because normal site-packages is not writeable
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/241.3 MB ? eta -:--:--
   ---------------------

In [34]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

finbert_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
finbert_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [42]:
def analyze_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "neutral"  # default fallback
    
    inputs = finbert_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = finbert_model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    labels = ["positive", "neutral", "negative"]
    return labels[probs.argmax().item()]

df_comments.dropna(subset=["Comment"], inplace=True)
df_comments["Sentiment"] = df_comments["Comment"].apply(lambda x: analyze_sentiment(x))

In [56]:
df_comments["date"] = df_comments["Created UTC"].dt.date

In [109]:
NVDA = yf.download("NVDA", period="5y", timeout=30)
NVDA = NVDA[["Close", "Volume"]]
NVDA["Return"] = NVDA["Close"].pct_change()

C:\Users\zihan\AppData\Local\Temp\ipykernel_9888\1029336495.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  NVDA = yf.download("NVDA", period="5y", timeout=30)
[*********************100%***********************]  1 of 1 completed


In [87]:
NVDA_sentiment = df_comments.groupby("date")["Sentiment"].value_counts().unstack(fill_value=0)
NVDA_sentiment[["positive","neutral","negative"]] = NVDA_sentiment[["positive","neutral","negative"]].div(NVDA_sentiment.sum(axis=1), axis=0)
NVDA_sentiment = NVDA_sentiment.reindex(NVDA.index.date, fill_value=0)
NVDA_sentiment

Sentiment,negative,neutral,positive
date,,,
2020-08-25,0.000000,0.000000,0.000000
2020-08-26,0.000000,0.000000,0.000000
2020-08-27,0.000000,0.000000,0.000000
2020-08-28,0.000000,0.000000,0.000000
2020-08-31,0.000000,0.000000,0.000000
...,...,...,...
2025-08-18,0.054054,0.270270,0.675676
2025-08-19,0.320000,0.080000,0.600000
2025-08-20,0.105263,0.052632,0.842105


In [113]:
df_reset = NVDA.reset_index()
df_reset.columns = df_reset.columns.get_level_values(-2)
df_reset = df_reset.rename(columns={'Date': 'date'})
NVDA = df_reset

In [146]:
NVDA_sentiment.reset_index(inplace=True)
NVDA_sentiment["date"] = pd.to_datetime(NVDA_sentiment["date"])

In [147]:
merged = pd.merge(NVDA, NVDA_sentiment, on="date", how="inner")
merged.tail()

,date,Close,Volume,Return,level_0,index,negative,neutral,positive
1250,2025-08-18,182.009995,132008000,0.008645,1250,1250,0.054054,0.270270,0.675676
1251,2025-08-19,175.639999,185229200,-0.034998,1251,1251,0.320000,0.080000,0.600000
1252,2025-08-20,175.399994,215142700,-0.001366,1252,1252,0.105263,0.052632,0.842105
1253,2025-08-21,174.979996,140040900,-0.002395,1253,1253,0.313725,0.039216,0.647059
1254,2025-08-22,177.990005,172334200,0.017202,1254,1254,0.187500,0.291667,0.520833


In [123]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.17.3-cp313-cp313-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.74.0-cp313-cp313-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.11.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Features and target
features = ["Close", "Volume", "Return", "positive", "neutral", "negative"]
target = "Return"

scaler = MinMaxScaler()
scaled = scaler.fit_transform(merged[features])

X, y = [], []
window = 30  # use past 30 days

for i in range(window, len(scaled)):
    X.append(scaled[i-window:i])
    y.append(scaled[i, features.index(target)])

X, y = np.array(X), np.array(y)
print(X.shape, y.shape)  # (samples, timesteps, features)


(1225, 30, 6) (1225,)


In [149]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dense(1)  # Predict next-day return
])

model.compile(optimizer="adam", loss="mse")
history = model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2, shuffle=False)


Epoch 1/20


C:\Users\zihan\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - loss: 0.0260 - val_loss: 0.0068
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0084 - val_loss: 0.0065
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0078 - val_loss: 0.0065
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0074 - val_loss: 0.0066
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0070 - val_loss: 0.0070
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0069 - val_loss: 0.0072
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0070 - val_loss: 0.0072
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0067 - val_loss: 0.0070
Epoch 9/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0068 - val_loss: 0.0069
Epoch 10/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0068 - val_loss: 0.0068
Epoch 11/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0067 - val_loss: 0.0066
Epoch 12/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0066 - val_l

In [150]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

tscv = TimeSeriesSplit(n_splits=5)

for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f"Fold {fold+1}")
    
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Build a fresh LSTM each fold
    model = Sequential([
        LSTM(50, activation="relu", input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")

    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    preds = model.predict(X_val).flatten()
    mse = np.mean((preds - y_val) ** 2)
    print("Validation MSE:", mse)



Fold 1
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step
Validation MSE: 0.01299391421637081
Fold 2
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
Validation MSE: 0.008510941951704905
Fold 3
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Validation MSE: 0.005946710861233794
Fold 4
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step
Validation MSE: 0.007765290241587716
Fold 5
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step
Validation MSE: 0.007948267234168508
